# Main analysis on api collected data to follow

In [127]:
# import libraries

import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import json
from pathlib import Path
import numpy as np
import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv

%matplotlib inline


In [128]:
#pn.extension('plotly')

In [129]:
import hvplot.pandas

In [130]:
# Read the API keys
load_dotenv()

True

In [131]:
# Create latest up to date dataset for each position reviewed:
# A historical dataset is created by a separate script that pulls, massages the data and creates a csv.
# This data is then pulled into a df and then updated with latest market information using appropriate API
# get request. This way historical data doesnt need to be retrieved with repetitive API gets, but the data 
# can be kept up to date with latest info.

# #each should return, cleaned properly and format
#def BondAgg_latest_data
#
#date adj close price
#
#def BTCUSD_latest_data
#
#def XAUUSD_latest_data

## Fetch ART Data ##
art_file_path = Path("../data_imports/Artprice_Indexes_Quarterly_data_Base100_January1998.csv")
art_index = pd.read_csv(art_file_path, encoding='utf-8', parse_dates=True, infer_datetime_format=True)
art_index.rename(columns=({"Unnamed: 0" : "Date"}), inplace=True)
art_index.dropna()
art_index.set_index("Date", drop=True, inplace=True)
#art_index.tail()

In [132]:
## Fetch SP500 Data ##
ms_api_key = os.getenv("MARKETSTACK_API_KEY")
ms_api_key_string = "&access_key=" + ms_api_key 
print(type(ms_api_key))
ms_request_url = "https://api.marketstack.com/v1/eod?symbols=GSPC.INDX"
ms_request_url = ms_request_url + ms_api_key_string + "&limit=3000" 
# Execute get request with API key
ms_response_data = requests.get(ms_request_url)
ms_data = ms_response_data.json()
#print(json.dumps(data, indent=4))
sp_list = ms_data["data"]
# Pull json outout into dataframe and update the adjusted closing prices into columns
column_list = ['Date', 'Position', 'AdjClose']
sp500_df = pd.DataFrame(columns=column_list)
for dict in sp_list: 
    for list in dict: 
        if list == "date":
            temp_date = str(dateutil.parser.parse(dict[list]).date())
            #print(temp_date)
        elif list == "adj_close":
            temp_close = dict[list] 
            
    new_row = {'Date':temp_date, 'Position':'SP500', 'AdjClose':temp_close}
    sp500_df = sp500_df.append(new_row, ignore_index=True)                 
    
sp500_df["Date"] = pd.to_datetime(sp500_df["Date"])
sp500_df.set_index("Date", drop=True, inplace=True)
#sp500_df.rename(columns = {"sp500_adj_closing" : "Price"}, inplace=True)

sp500_df.head()


<class 'str'>


Position   AdjClose
Date                          
2020-11-04    SP500  3443.4399
2020-11-03    SP500  3369.1599
2020-11-02    SP500  3310.2400
2020-10-30    SP500  3269.9600
2020-10-29    SP500  3310.1101

In [133]:
## Fetch crypto Data ##
api_key = os.getenv("NOMICS_API_KEY")
print(type(api_key))
api_key_string = "key=" + api_key
request_url = "https://api.coindesk.com/v1/bpi/historical/close.json?start=2012-01-01&end=2020-10-05"
# Execute get request
response_data = requests.get(request_url)
data = response_data.json()
#print(json.dumps(data, indent=4))
bpi_list = data["bpi"]
#print(json.dumps(bpi_list, indent=4))
bpi_df = pd.DataFrame.from_dict(bpi_list, orient='index',
                       columns=['bpi_closing'])
#bpi_df.columns = ['bpi_closing']
#bpi_df.set_index(bpi_df['date'],inplace=True)
bpi_df.head()

<class 'str'>


bpi_closing
2012-01-01       5.2677
2012-01-02       5.2168
2012-01-03       4.8808
2012-01-04       5.5738
2012-01-05       6.9476

In [134]:
# first select art index of choice and scale up price by 100 & add position name column
gblart_index_col_list = ['Position','AdjClose']
gblart_index_df = pd.DataFrame(columns=gblart_index_col_list)
gblart_index_df['AdjClose'] = (art_index.loc[:,'Global Index (USD)'])*100
gblart_index_df['Position'] = 'GlobalArtIndex'
gblart_index_df.head()


Position  AdjClose
Date                                
1998-01-01  GlobalArtIndex     10000
1998-04-01  GlobalArtIndex     10700
1998-07-01  GlobalArtIndex     10600
1998-10-01  GlobalArtIndex      9600
1999-01-01  GlobalArtIndex     10400

In [135]:
####################################################################
# EVERYTHING ABOVE WILL BE REVISED ONCE DATASETS FUNCTIONS CREATED
# BELOW STARTS THE FINAL VERSION OF ANALYTICS AND VISUALIZATIONS
####################################################################
# 
# THIS CELL TEMPORARILY COMMENTED OUT TO QUICK HACK SOMETHING FOR KRIS PLOTS
#
####################################################################

# Define positions (indexes, algorithmitic portfolios, assets to evaluate
# Define currencies to review performance against
# Define evaluation calculations to perform
# All are manually coded in for first MVP
#eval_pos_df_list = [
#                #'FAANG',
#                #'BondAgg',
#                #'Resi_RE',
#                #'GlobalArtIndex',
#                sp500_df
#                ]
#currency_df_list = [
#                USD,
#                BTCUSD,
#                XAUUSD
#                ]
#calc_list = ['price','returns_daily','returns_cum','std']
#
#
#calc_dict = {
#            pos : [pos_price_USD_df,pos_price_]
#}


In [136]:
####################################################################
# 
# THIS CELL TEMPORARILY COMMENTED OUT TO QUICK HACK SOMETHING FOR KRIS PLOTS
#
####################################################################
#def sanitize_dataset(dataset_df):
#    """Insure data sorted ascending"""
#    dataset_df = dataset_df.sort_index()
#    print(dataset_df.head())
#    return(dataset_df)
#
## Sweep each position,for each currency, for each calculation
#for pos in eval_pos_list:
#    # first do finalize sanitation step on incoming position dataset
#    print(type(pos))
#    sanitize_dataset(pos)
#    for currency in currency_df_list:
#        if (currency == USD):
# 

In [137]:
# Create long-form USD price df for all pos
price_frames = [
                #'FAANG',
                #'BondAgg',
                #'Resi_RE',
                gblart_index_df,
                sp500_df
                ]                    

lcombo_USD_df = pd.concat(price_frames)
#lcombo_USD_df

lcombo_USD_df['Returns'] = lcombo_USD_df['AdjClose'].pct_change()
#lcombo_USD_df

#lcombo_USD_df['STD'] = lcombo_USD_df['AdjClose'].groupby('Position').std()
lcombo_USD_df['STD'] = lcombo_USD_df['AdjClose'].std()
lcombo_USD_df   

Position    AdjClose   Returns          STD
Date                                                                  
1998-01-01           GlobalArtIndex  10000.0000       NaN  2668.272716
1998-04-01           GlobalArtIndex  10700.0000  0.070000  2668.272716
1998-07-01           GlobalArtIndex  10600.0000 -0.009346  2668.272716
1998-10-01           GlobalArtIndex   9600.0000 -0.094340  2668.272716
1999-01-01           GlobalArtIndex  10400.0000  0.083333  2668.272716
1999-04-01           GlobalArtIndex  10300.0000 -0.009615  2668.272716
1999-07-01           GlobalArtIndex  10400.0000  0.009709  2668.272716
1999-10-01           GlobalArtIndex  10400.0000  0.000000  2668.272716
2000-01-01           GlobalArtIndex  11300.0000  0.086538  2668.272716
2000-04-01           GlobalArtIndex  11600.0000  0.026549  2668.272716
2000-07-01           GlobalArtIndex  11200.0000 -0.034483  2668.272716
2000-10-01           GlobalArtIndex  11500.0000  0.026786  2668.272716
2001-01-01           GlobalArtIndex  10400.0000 -0.095652  2668.272716
2001-04-01           GlobalArtIndex  12200.0000  0.173077  2668.272716
2001-07-01           GlobalArtIndex  10700.0000 -0.122951  2668.272716
2001-10-01           GlobalArtIndex  10700.0000  0.000000  2668.272716
2002-01-01           GlobalArtIndex  10700.0000  0.000000  2668.272716
2002-04-01           GlobalArtIndex  11000.0000  0.028037  2668.272716
2002-07-01           GlobalArtIndex  11000.0000  0.000000  2668.272716
2002-10-01           GlobalArtIndex  11200.0000  0.018182  2668.272716
2003-01-01           GlobalArtIndex  12700.0000  0.133929  2668.272716
2003-04-01           GlobalArtIndex  14000.0000  0.102362  2668.272716
2003-07-01           GlobalArtIndex  14000.0000  0.000000  2668.272716
2003-10-01           GlobalArtIndex  12200.0000 -0.128571  2668.272716
2004-01-01           GlobalArtIndex  14200.0000  0.163934  2668.272716
2004-04-01           GlobalArtIndex  15500.0000  0.091549  2668.272716
2004-07-01           GlobalArtIndex  15400.0000 -0.006452  2668.272716
2004-10-01           GlobalArtIndex  15500.0000  0.006494  2668.272716
2005-01-01           GlobalArtIndex  16300.0000  0.051613  2668.272716
2005-04-01           GlobalArtIndex  18700.0000  0.147239  2668.272716
...                             ...         ...       ...          ...
2010-12-17 00:00:00           SP500   1243.9100 -0.002542  2668.272716
2010-12-16 00:00:00           SP500   1242.8700 -0.000836  2668.272716
2010-12-15 00:00:00           SP500   1235.2300 -0.006147  2668.272716
2010-12-14 00:00:00           SP500   1241.5900  0.005149  2668.272716
2010-12-13 00:00:00           SP500   1240.4600 -0.000910  2668.272716
2010-12-10 00:00:00           SP500   1240.4000 -0.000048  2668.272716
2010-12-09 00:00:00           SP500   1233.0000 -0.005966  2668.272716
2010-12-08 00:00:00           SP500   1228.2800 -0.003828  2668.272716
2010-12-07 00:00:00           SP500   1223.7500 -0.003688  2668.272716
2010-12-06 00:00:00           SP500   1223.1200 -0.000515  2668.272716
2010-12-03 00:00:00           SP500   1224.7100  0.001300  2668.272716
2010-12-02 00:00:00           SP500   1221.5300 -0.002597  2668.272716
2010-12-01 00:00:00           SP500   1206.0699 -0.012656  2668.272716
2010-11-30 00:00:00           SP500   1180.5500 -0.021160  2668.272716
2010-11-29 00:00:00           SP500   1187.7600  0.006107  2668.272716
2010-11-26 00:00:00           SP500   1189.4000  0.001381  2668.272716
2010-11-24 00:00:00           SP500   1198.3500  0.007525  2668.272716
2010-11-23 00:00:00           SP500   1180.7300 -0.014704  2668.272716
2010-11-22 00:00:00           SP500   1197.8400  0.014491  2668.272716
2010-11-19 00:00:00           SP500   1199.7300  0.001578  2668.272716
2010-11-18 00:00:00           SP500   1196.6899 -0.002534  2668.272716
2010-11-17 00:00:00           SP500   1178.5900 -0.015125  2668.272716
2010-11-16 00:00:00           SP500   1178.3400 -0.000212  2668.272716
2010-11-15 00:00:00           SP500   1197.7500  0.

In [138]:
# HENRY SCRATCHPAD

#for currency in currency_list:
#    for pos in eval_pos_list:
#        # grab pos dataset and insure arranged ascending
#        calc_df = {pos}
       
# calculate for each pos in each currency then concat at end as combo_calc_currency
#benchmk_pos_list = ['SP500','GlobalArtIndex']
#
#for pos in benchmk_pos_list:
 #   print(f"the ticker is {pos}")

#test_price_df = pd.DataFrame(columns=benchmk_pos_list)
#test_price_df = sp500_df
#
#test_price_df =  test_price_df.merge(
#                                    gblart_index_df, 
#                                    how='left',
#                                    left_index=True,
#                                    right_index=True
#                                    )
#
#test_returns_df = test_price_df.pct_change() #
#test_std_df = test_price_df.std()
#test_std_df
#
#

#sp500_daily_returns = sp500_df.pct_change().dropna()
#sp500_daily_returns.head()

In [139]:
#gblart_returns_df = pd.DataFrame(columns=['GlobalArtIndex'])
#gblart_returns_df['GlobalArtIndex'] = art_index.loc[:,'Global Index (USD)'].pct_change().dropna()
#test_combo_plot = gblart_returns_df.hvplot.line(title="test combo Returns", ylabel="Daily Returns", frame_width=900, frame_height=450, rot=90)
#test_combo_plot
#gblart_index_df = pd.DataFrame(columns=['GlobalArtIndex'])



In [140]:
#returns_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#cum_returns_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#std_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#sharpe_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)

#returns_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#cum_returns_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#std_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#sharpe_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)

#returns_eval_10yr_df['GlobalArtIndex'] = art_index.loc[:,'Global Index (USD)'].pct_change().dropna()
#returns_eval_10yr_df.head()

# test merge trying zero fill NaN locations to help with line plot below -- spoiler alert -- doesnt work
#test_combo_df = sp500_daily_returns.merge(gblart_returns_df,how='left',left_index=True,right_index=True).fillna(0)
# long head here to verify that its joining where there's valid art data
#test_combo_df.head(100)


In [141]:
#test_combo_plot = test_combo_df.hvplot.line(title="test combo ", ylabel="Returns", frame_width=900, frame_height=450, rot=90)
#test_combo_plot

In [8]:
####################################################################
# THIS CELL IS START OF VISUALIZATION FUNCTIONS 
#
####################################################################
# Define Panel Visualization Functions
# Number of plot functions right now = 5 calcs/position * 2timeframes * 6 positions = 60!!!!


#def 
#art_index_std_bar = px.bar(art_index_std, title="Art Price STD 1998-2020")
#art_index_std_bar

In [2]:
# Create Panel Dashboard

